In [146]:
import pandas as pd
from goodreads import client

In [147]:
books = pd.read_csv('good_books_10k/good_books_long.csv')

In [159]:
books[books['original_title'] == 'So Long, and Thanks for All the Fish']

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
1013,8698,Douglas Adams,1984.0,"So Long, and Thanks for All the Fish",4.08,624,4454,22611,39988,41858,Including everything you wanted to know about ...


In [8]:
goodreads_key = 't6mIkabukH29jAey0381yA'
goodreads_secret = '6IWvvO5CNFIAqUragee2Bb5HkEOvxIYqSXeXdFSHvM'
good_client = client.GoodreadsClient(goodreads_key, goodreads_secret)

In [0]:
# get descriptions for books in the dataset using the book_ids
import time
good_descriptions = []
i = 1
for book_id in books['goodreads_book_id']:
    try:
        print(i)
        i += 1
        book = good_client.book(book_id)
        good_descriptions.append(book.description)
        time.sleep(0.5)
    except:
        pass

In [0]:
books['description'] = pd.Series(good_descriptions)

In [9]:
books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,"Could you survive on your own, in the wild, wi..."
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,Harry Potter's life is miserable. His parents ...
2,41865,Stephenie Meyer,2005.0,Twilight,3.57,456191,436802,793319,875073,1355439,<b>About three things I was absolutely positiv...
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,The unforgettable novel of a childhood in a sl...
4,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,86236,197621,606158,936012,947718,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [75]:
# function to remove the 'Alternate Cover Edition ...' paragraph at the beginning 
def clean_cover_isbn(text):
  try:
    text = text.split('<br />')[2]
    return text
  except:
    return text

In [76]:
# function to remove html markings
from bs4 import BeautifulSoup
def clean_html(text):
  try:
    soup = BeautifulSoup(text, 'lxml')
    clean_text = soup.get_text()
    return clean_text
  except:
    return text

In [77]:
# function to remove punctuation
import string
def clean_punctuations(text):
  clean_text = "".join([c for c in text if c not in string.punctuation])
  return clean_text

In [0]:
# somehow there were books that were skipped during the first scrape, so here we go again
retrieve_books = books[books['description'].isnull()]
retrieve_books.reset_index(drop = True, inplace = True)

In [39]:
retrieve_books.shape

(125, 11)

In [0]:
import time
retrieve_good_descriptions = []
i = 1
for book_id in retrieve_books['goodreads_book_id']:
    try:
        print(i)
        i += 1
        book = good_client.book(book_id)
        retrieve_good_descriptions.append(book.description)
        time.sleep(0.5)
    except:
        pass

In [0]:
retrieve_good_descriptions

In [0]:
retrieve_books['description'] = pd.Series(retrieve_good_descriptions)

In [0]:
retrieve_books = retrieve_books[~retrieve_books['description'].isnull()]

In [62]:
books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,"Could you survive on your own, in the wild, wi..."
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,Harry Potter's life is miserable. His parents ...
2,41865,Stephenie Meyer,2005.0,Twilight,3.57,456191,436802,793319,875073,1355439,<b>About three things I was absolutely positiv...
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,The unforgettable novel of a childhood in a sl...
4,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,86236,197621,606158,936012,947718,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [0]:
info_books = books[~books['description'].isnull()]

In [0]:
info_books = pd.concat([info_books, retrieve_books])

In [75]:
info_books.shape

(9979, 11)

In [0]:
info_books['description'] = info_books['description'].apply(lambda x: clean_cover_isbn(x))
info_books['description'] = info_books['description'].apply(lambda x: clean_html(x))
info_books['description'] = info_books['description'].apply(lambda x: clean_punctuations(x))

In [77]:
info_books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,In the ruins of a place once known as North Am...
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,After a lifetime of bottling up his magical po...
2,41865,Stephenie Meyer,2005.0,Twilight,3.57,456191,436802,793319,875073,1355439,First Edward was a vampire
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,Compassionate dramatic and deeply moving To Ki...
4,4671,F. Scott Fitzgerald,1925.0,The Great Gatsby,3.89,86236,197621,606158,936012,947718,The Great Gatsby F Scott Fitzgeralds third boo...


In [0]:
info_books['description'] = info_books['description'].apply(lambda x: get_tokens(x))

In [44]:
info_books = info_books[info_books['description'].map(lambda x: len(x) > 2)]

In [ ]:
info_books.to_csv('good_books_10k/good_books_tokenized.csv', index = False)

In [13]:
info_books.to_csv('good_books_en.csv', index = False)

Ordered from here

In [124]:
import pandas as pd
info_books = pd.read_csv('good_books_10k/good_books_long.csv')

In [125]:
def check_alphabet(text):
    try:
        return text[0][0] in 'abcdefghijklmnopqrstuvwxyz'.upper()
    except:
        return False

In [126]:
info_books['check_alphabet'] = info_books['description'].apply(lambda x: check_alphabet(x))

In [127]:
info_books = info_books[info_books['check_alphabet'] == True]

In [128]:
def check_alternate(text):
    return text.split(' ')[:3] == ['Alternate', 'Cover', 'Edition']

In [129]:
info_books['check_alternate'] = info_books['description'].apply(lambda x: check_alternate(x))
info_books = info_books[info_books['check_alternate'] == False]

In [130]:
info_books['description'] = info_books['description'].apply(lambda x: clean_cover_isbn(x))
info_books['description'] = info_books['description'].apply(lambda x: clean_html(x))
info_books['description'] = info_books['description'].apply(lambda x: clean_punctuations(x))

In [131]:
info_books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description,check_alphabet,check_alternate
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,In the ruins of a place once known as North Am...,True,False
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,After a lifetime of bottling up his magical po...,True,False
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,Compassionate dramatic and deeply moving To Ki...,True,False
5,11870085,John Green,2012.0,The Fault in Our Stars,4.26,47994,92723,327550,698471,1311871,Insightful bold irreverent and raw The Fault i...,True,False
7,5107,J.D. Salinger,1951.0,The Catcher in the Rye,3.79,109383,185520,455042,661516,709176,JD Salingers classic novel of teenage angst an...,True,False


In [88]:
import nltk
from nltk.tokenize import word_tokenize

In [136]:
def get_tokens(text):
    return word_tokenize(text)

In [137]:
info_books['description'] = info_books['description'].apply(lambda x: get_tokens(x))

In [103]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [139]:
def clean_stop_words(text):
    return [word for word in text if word not in stop_words]

In [140]:
info_books['description'] = info_books['description'].apply(lambda x: clean_stop_words(x))

In [142]:
info_books.head()

,goodreads_book_id,authors,original_publication_year,original_title,average_rating,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,description,check_alphabet,check_alternate
0,2767052,Suzanne Collins,2008.0,The Hunger Games,4.34,66715,127936,560092,1481305,2706317,"[In, ruins, place, known, North, America, lies...",True,False
1,3,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,4.44,75504,101676,455024,1156318,3011543,"[After, lifetime, bottling, magical, powers, H...",True,False
3,2657,Harper Lee,1960.0,To Kill a Mockingbird,4.25,60427,117415,446835,1001952,1714267,"[Compassionate, dramatic, deeply, moving, To, ...",True,False
5,11870085,John Green,2012.0,The Fault in Our Stars,4.26,47994,92723,327550,698471,1311871,"[Insightful, bold, irreverent, raw, The, Fault...",True,False
7,5107,J.D. Salinger,1951.0,The Catcher in the Rye,3.79,109383,185520,455042,661516,709176,"[JD, Salingers, classic, novel, teenage, angst...",True,False


In [145]:
info_books.to_csv('good_books_10k/good_books_en.csv', index = False)

In [144]:
info_books.shape

(7346, 13)